In [ ]:
# %pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# %pip install transformers[torch]==4.30.2
# %pip install accelerate -U
# %pip install optuna
%pip install ipywidgets

In [14]:
import ipywidgets as widgets

In [ ]:
model_choice_int = 1
bert_model_path = 'indolem/indobert-base-uncased'
distilbert_model_path = 'cahya/distilbert-base-indonesian'

# 0 = distilbert, 1 = bert

widgets.Dropdown(
    options=[('Distilbert', 1), ('Indobert', 2)],
    value=1,
    description='Model:',
    disabled=False,
    
)

Dropdown(description='Model:', options=(('Distilbert', 1), ('Indobert', 2)), value=1)

In [3]:
import pandas as pd

datasets = {}

df = pd.read_csv('Dataset\Pre-Processed Dataset\combined_dataset.csv')
datasets['combined_dataset'] = df[['encoded_label', 'clean_text']]

datasets['combined_dataset'].head(10)

,encoded_label,clean_text
0,1.0,kaka tidur yaa sudah pagi tidak boleh capek2
1,1.0,makan nasi padang saja badannya
2,0.0,suka cukur jembut manggung
3,1.0,hai kak isyana ngefans sekali kak isyana suka ...
4,1.0,manusia bidadari sih herann deh cantik
5,0.0,ayu kinantii isyan sekarang berubah ya baju ny...
6,1.0,gemesnya isyan mirip tango berlapis lapis ciaaaa
7,0.0,jelek saja anaknya ayahnya cakep2
8,0.0,anaknya mirip sudah tua begitu ya mukanya kart...
9,0.0,muka anak nya ko tua sekali yaa tidak ngegemes...


## Train-Test Split

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)

# Load tokenizer
if model_choice_int == 0:
    tokenizer = AutoTokenizer.from_pretrained(distilbert_model_path)
    
else:
    tokenizer = AutoTokenizer.from_pretrained(bert_model_path)


# Dictionary to hold train and test custom datasets
train_test_datasets = {}

for name, df in datasets.items():
    # Get text and labels
    X = df['clean_text'].tolist()
    y = df['encoded_label'].tolist()

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Tokenize
    train_encodings = tokenizer(X_train, truncation=True, padding=True)
    test_encodings = tokenizer(X_test, truncation=True, padding=True)

    # Create datasets
    train_dataset = CustomDataset(train_encodings, y_train)
    test_dataset = CustomDataset(test_encodings, y_test)

    # Store in Dict
    train_test_datasets[name] = {
        "train": train_dataset,
        "test": test_dataset,
        # "val": val_dataset

    }

    print(f"Dataset: {name}, Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")

c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset: combined_dataset, Train size: 3033, Test size: 759


## Train and Evaluation


In [6]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [7]:
output_dir = f'./results/{name}'
log_dir = f'./logs/{name}'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

default_training_args = TrainingArguments(
    output_dir=output_dir,
    run_name=f'training_{name}',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # gradient_accumulation_steps=2,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir=log_dir,
    logging_steps=10,
    do_eval=True,
    eval_steps=500,
    save_strategy="epoch",
    save_total_limit=1,
    report_to=[],
    fp16=True,
    
)

best_training_args_distil = TrainingArguments(
    output_dir=output_dir,
    run_name=f'training_{name}',
    per_device_eval_batch_size=16,  # can stay as-is for evaluation
    num_train_epochs=6,            # ⬅️ from Optuna
    per_device_train_batch_size=16,  # ⬅️ from Optuna
    warmup_steps=29,               # ⬅️ from Optuna
    weight_decay=0.09793901282245424,  # ⬅️ from Optuna
    learning_rate=3.694163912198525e-05,  # ⬅️ from Optuna
    logging_dir=log_dir,
    logging_steps=10,
    do_eval=True,
    eval_steps=500,
    save_strategy="epoch",
    save_total_limit=1,
    report_to=[],
    fp16=True,
)

best_training_args_bert = TrainingArguments(
    output_dir=output_dir,
    run_name=f'training_{name}',
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,  # From Optuna
    num_train_epochs=9,             # From Optuna
    warmup_steps=117,               # From Optuna
    weight_decay=0.12399629519542921,  # From Optuna
    learning_rate=2.9677882858655988e-05,  # From Optuna
    logging_dir=log_dir,
    logging_steps=10,
    do_eval=True,
    eval_steps=500,
    save_strategy="epoch",
    save_total_limit=1,
    report_to=[],
    fp16=True,
)


### Default


In [8]:
os.environ["WANDB_MODE"] = "disabled"

In [9]:
def evaluate_model(model, test_dataset, batch_size, output_dir, dataset_name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    dataloader = DataLoader(test_dataset, batch_size=batch_size)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            # Move inputs to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Move to CPU and accumulate
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Metrics
    report = classification_report(all_labels, all_preds, digits=4)
    with open(os.path.join(output_dir, "classification_report.txt"), "w") as f:
        f.write(report)

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix: {dataset_name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "confusion_matrix.png"))
    plt.close()

    print(f"✅ Evaluation complete for {dataset_name}")

In [10]:
print(f"\n🚀 Training on dataset: {name}")

# Initialize model
if model_choice_int == 0:
    model = DistilBertForSequenceClassification.from_pretrained(
            distilbert_model_path, num_labels=2
    )

    training_args = best_training_args_distil

elif model_choice_int == 1:
    model = BertForSequenceClassification.from_pretrained(
            bert_model_path, num_labels=2
    )

    training_args = best_training_args_bert

else:
    model = DistilBertForSequenceClassification.from_pretrained(
            distilbert_model_path, num_labels=2
    )

    training_args = default_training_args


print(training_args)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test_datasets[name]['train'],
    eval_dataset=train_test_datasets[name]['test'],
)

trainer.train()

# Save model
trainer.save_model(output_dir)


🚀 Training on dataset: combined_dataset


c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowliste

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HU

c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3420 [00:00<?, ?it/s]

{'loss': 0.6858, 'learning_rate': 1.7755998291503582e-06, 'epoch': 0.03}
{'loss': 0.7449, 'learning_rate': 4.31217101365087e-06, 'epoch': 0.05}
{'loss': 0.6916, 'learning_rate': 6.59508507970133e-06, 'epoch': 0.08}
{'loss': 0.7761, 'learning_rate': 9.131656264201844e-06, 'epoch': 0.11}
{'loss': 0.7092, 'learning_rate': 1.1668227448702355e-05, 'epoch': 0.13}
{'loss': 0.7449, 'learning_rate': 1.3951141514752815e-05, 'epoch': 0.16}
{'loss': 0.7206, 'learning_rate': 1.6234055580803276e-05, 'epoch': 0.18}
{'loss': 0.7211, 'learning_rate': 1.8770626765303786e-05, 'epoch': 0.21}
{'loss': 0.7213, 'learning_rate': 2.13071979498043e-05, 'epoch': 0.24}
{'loss': 0.7765, 'learning_rate': 2.384376913430481e-05, 'epoch': 0.26}
{'loss': 0.639, 'learning_rate': 2.638034031880532e-05, 'epoch': 0.29}
{'loss': 0.6782, 'learning_rate': 2.8916911503305833e-05, 'epoch': 0.32}
{'loss': 0.695, 'learning_rate': 2.9614986951901344e-05, 'epoch': 0.34}
{'loss': 0.6061, 'learning_rate': 2.9525135656537564e-05, 'epo

In [11]:
# Evaluate and log
evaluate_model(model, train_test_datasets[name]['test'], 4, output_dir, name)

Evaluating: 100%|██████████| 190/190 [00:04<00:00, 43.02it/s]


✅ Evaluation complete for combined_dataset


### Optuna

In [12]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import Trainer, DistilBertForSequenceClassification, BertForSequenceClassification
import gc
from torch.utils.data import DataLoader
from tqdm import tqdm
from optuna import create_study
from sklearn.metrics import f1_score

def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 10),
        "warmup_steps": trial.suggest_int("warmup_steps", 0, 300),
    }

def manual_evaluate(model, dataset, batch_size=8):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")
    dataloader = DataLoader(dataset, batch_size=batch_size)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = {k: v.to(model.device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
            labels = batch["labels"].to(model.device)

            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = f1_score(all_labels, all_preds)

    torch.cuda.empty_cache()
    gc.collect()

    return acc
    

# Subclass Trainer to inject memory cleanup
class CleanTrainer(Trainer):
    def train(self, *args, **kwargs):
        result = super().train(*args, **kwargs)
        torch.cuda.empty_cache()
        gc.collect()
        return result

if model_choice_int == 0:
    # Initialize model
    model_init = lambda: DistilBertForSequenceClassification.from_pretrained(
        distilbert_model_path, num_labels=2
    )

else:
    model_init = lambda: BertForSequenceClassification.from_pretrained(
        bert_model_path, num_labels=2
    )



# Initialize the Trainer
trainer = CleanTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_test_datasets[name]["train"],
    eval_dataset=train_test_datasets[name]["test"],
    compute_metrics= manual_evaluate(trainer.model, train_test_datasets[name]['test'], batch_size=8),
)

def objective(trial):
    hp = optuna_hp_space(trial)

    training_args = TrainingArguments(
        output_dir=f"./results/optuna_trial_{trial.number}",
        num_train_epochs=hp["num_train_epochs"],
        per_device_train_batch_size=hp["per_device_train_batch_size"],
        warmup_steps=hp["warmup_steps"],
        weight_decay=hp["weight_decay"],
        save_strategy="no",        # ← disables saving
        save_total_limit=0,         # ← just in case (optional safety)
        logging_dir=f"./logs/optuna_trial_{trial.number}",
        report_to=[],  # Disable wandb
        fp16=True,
    )

    model = model_init()

    trainer = CleanTrainer(
        model=model,
        args=training_args,
        train_dataset=train_test_datasets[name]["train"],
        eval_dataset=train_test_datasets[name]["test"],
    )

    trainer.train()

    acc = manual_evaluate(trainer.model, train_test_datasets[name]["test"], batch_size=8)
    return acc

study = create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("✅ Best trial:")
print(study.best_trial)



Evaluating: 100%|██████████| 95/95 [00:02<00:00, 41.57it/s]
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to b

  0%|          | 0/1520 [00:00<?, ?it/s]

{'loss': 0.463, 'learning_rate': 3.5904628330995795e-05, 'epoch': 1.32}
{'loss': 0.1532, 'learning_rate': 1.8373071528751755e-05, 'epoch': 2.63}
{'loss': 0.053, 'learning_rate': 8.415147265077139e-07, 'epoch': 3.95}
{'train_runtime': 130.8412, 'train_samples_per_second': 92.723, 'train_steps_per_second': 11.617, 'train_loss': 0.22079304056732277, 'epoch': 4.0}


Evaluating: 100%|██████████| 95/95 [00:01<00:00, 55.88it/s]
[I 2025-08-18 18:48:26,155] Trial 0 finished with value: 0.9654088050314465 and parameters: {'learning_rate': 2.4111941151269226e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.2871265242030436, 'num_train_epochs': 4, 'warmup_steps': 94}. Best is trial 0 with value: 0.9654088050314465.
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (Se

  0%|          | 0/1520 [00:00<?, ?it/s]

...

Evaluating: 100%|██████████| 95/95 [00:01<00:00, 56.95it/s]
[I 2025-08-18 20:48:55,226] Trial 28 finished with value: 0.9795918367346939 and parameters: {'learning_rate': 3.217732490546123e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.03351354423237021, 'num_train_epochs': 5, 'warmup_steps': 186}. Best is trial 14 with value: 0.9922239502332814.
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling 

  0%|          | 0/3800 [00:00<?, ?it/s]

{'loss': 0.5101, 'learning_rate': 4.63684800897364e-05, 'epoch': 1.32}
{'loss': 0.2169, 'learning_rate': 3.935782389231632e-05, 'epoch': 2.63}
{'loss': 0.0893, 'learning_rate': 3.2347167694896246e-05, 'epoch': 3.95}
{'loss': 0.0633, 'learning_rate': 2.533651149747617e-05, 'epoch': 5.26}
{'loss': 0.0488, 'learning_rate': 1.8325855300056086e-05, 'epoch': 6.58}
{'loss': 0.0298, 'learning_rate': 1.1315199102636007e-05, 'epoch': 7.89}
{'loss': 0.0326, 'learning_rate': 4.304542905215928e-06, 'epoch': 9.21}
{'train_runtime': 315.7421, 'train_samples_per_second': 96.059, 'train_steps_per_second': 12.035, 'train_loss': 0.13254492232674048, 'epoch': 10.0}


Evaluating: 100%|██████████| 95/95 [00:02<00:00, 47.10it/s]
[I 2025-08-18 20:54:14,619] Trial 29 finished with value: 0.9761526232114467 and parameters: {'learning_rate': 4.258744059975119e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.2639083181574777, 'num_train_epochs': 10, 'warmup_steps': 234}. Best is trial 14 with value: 0.9922239502332814.


✅ Best trial:
FrozenTrial(number=14, state=TrialState.COMPLETE, values=[0.9922239502332814], datetime_start=datetime.datetime(2025, 8, 18, 19, 39, 9, 336919), datetime_complete=datetime.datetime(2025, 8, 18, 19, 44, 3, 943244), params={'learning_rate': 3.164159687360286e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.1640557421098752, 'num_train_epochs': 10, 'warmup_steps': 300}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=5e-05, log=True, low=1e-05, step=None), 'per_device_train_batch_size': CategoricalDistribution(choices=(4, 8, 16)), 'weight_decay': FloatDistribution(high=0.3, log=False, low=0.0, step=None), 'num_train_epochs': IntDistribution(high=10, log=False, low=2, step=1), 'warmup_steps': IntDistribution(high=300, log=False, low=0, step=1)}, trial_id=14, value=None)


In [13]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer  # Only if tokenizer not yet loaded


# 1. Retrieve best hyperparameters from Optuna
best_hp = study.best_trial.params

# 2. Re-initialize training arguments with best parameters
final_training_args = TrainingArguments(
    output_dir="./results/final_eval",
    num_train_epochs=best_hp["num_train_epochs"],
    per_device_train_batch_size=best_hp["per_device_train_batch_size"],
    warmup_steps=best_hp["warmup_steps"],
    weight_decay=best_hp["weight_decay"],
    learning_rate=best_hp["learning_rate"],
    save_strategy="no",
    report_to=[],
    fp16=True,
    logging_dir="./logs/final_eval"
)

# 3. Re-initialize model with same init function
final_model = model_init()

# 4. Retrain model on TRAIN + VAL before final test evaluation (optional but recommended)
from torch.utils.data import ConcatDataset
train_val_dataset = ConcatDataset([
    train_test_datasets[name]["train"],
    train_test_datasets[name]["val"]
])

# Load the same tokenizer used to tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained(distilbert_model_path if model_choice_int == 0 else bert_model_path)

# Create a data collator to dynamically pad batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


final_trainer = CleanTrainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_val_dataset,
    data_collator=data_collator  # <- Fixes padding issue
)

# 5. Train on full train+val
final_trainer.train()

# 6. Final evaluation on the test set
final_f1 = manual_evaluate(final_model, train_test_datasets[name]["test"], batch_size=8)

print(f"\n🎯 Final Test F1 Score (macro): {final_f1:.4f}")


c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\willi\miniconda3\envs\tGPU3.8\lib\site-packages\transformers\modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowliste

KeyError: 'val'